In [1]:
# from sim_configuration import *
from sim_cafe import *

sim = CafeSimulation("sim_config.yaml")
sim.create_ui()

In [ ]:
# TEST CYKLU ZÁKAZNÍKŮ V RŮZNÝCH ČASECH

from test_functions import *
from sim_configuration import ConfigurationManager

# Načti konfiguraci
config = ConfigurationManager.load_yaml("sim_config.yaml")

print("=" * 80)
print("FÁZE 1: TEST CYKLU ZÁKAZNÍKŮ V RŮZNÝCH ČASECH")
print("=" * 80)

# Definuj časy k testování (různé intervaly)
test_times = [
    450,  # Ranní špička
    500,  # Stále ranní špička
    650,  # Dopoledne
    750,  # Polední nápor
    900,  # Odpoledne
    1100,  # Večerní klid
]

for current_time in test_times:
    print("\n" + "=" * 80)
    print(f"⏰ ČAS: {current_time} minut ({current_time//60}:{current_time%60:02d})")
    print("=" * 80)

    # Test 1: Výběr intervalu
    interval = select_interval(config, current_time)

    if interval is None:
        print("❌ Žádný aktivní interval v tomto čase")
        continue

    print(f"\n📅 Interval: {interval.get_label()}")
    print(f"   Příchody/hod: {interval.arrival_rate.value}")
    print(f"   Váhy zákazníků: {interval.customer_mix.value}")

    # Vygeneruj několik skupin pro tento čas
    num_groups = 3
    print(f"\n👥 Generuji {num_groups} skupiny:")

    for group_num in range(num_groups):
        print(f"\n  --- Skupina {group_num + 1} ---")

        # Vyber typ zákazníka
        ctype_id = select_customer_type(interval)
        ctype_label = config.customer_types.__dict__.get(str(ctype_id)).get_label()
        print(f"  Typ: {ctype_label} (ID {ctype_id})")

        # Parametry skupiny
        params = generate_group_parameters(config, ctype_id)
        print(f"  Velikost: {params['size']} osob")
        print(f"  Trpělivost: {params['patience']:.2f} min")
        print(f"  Chce stůl: {'ANO' if params['wants_table'] else 'NE'}")
        print(f"  Rychlost konzumace: {params['consumption_modifier']:.2f}×")

        # Objednávky pro každého člena
        print(f"  Objednávky:")
        for member_num in range(params["size"]):
            order = create_order_for_customer(config, ctype_id)
            items_names = [
                config.menu_items.__dict__.get(str(iid)).get_label() for iid in order
            ]
            print(
                f"    Člen {member_num + 1}: {', '.join(items_names) if items_names else '(nic)'}"
            )

print("\n" + "=" * 80)
print("✅ TESTY DOKONČENY")
print("=" * 80)

In [ ]:
# TEST TYPŮ ZÁKAZNÍKŮ

from test_functions import *
from sim_configuration import ConfigurationManager

config = ConfigurationManager.load_yaml("sim_config.yaml")

print("=" * 80)
print("TEST VŠECH TYPŮ ZÁKAZNÍKŮ")
print("=" * 80)

# Projdi všechny typy zákazníků
for ctype_id_str, ctype_node in config.customer_types.items():
    if ctype_id_str == "label" or str(ctype_id_str).startswith("_"):
        continue

    ctype_id = int(ctype_id_str)

    print(f"\n{'='*80}")
    print(f"📋 TYP: {ctype_node.get_label()} (ID {ctype_id})")
    print(f"{'='*80}")

    # Konfigurace tohoto typu
    print("\n⚙️ Konfigurace:")
    print(f"   Váhy kategorií: {ctype_node.order_categories_preferences.value}")

    # Vygeneruj 10 skupin pro tento typ
    print("\n👥 Test 10 skupin:")

    for i in range(10):
        params = generate_group_parameters(config, ctype_id)
        print(
            f"   Skupina {i+1}: "
            f"size={params['size']}, "
            f"patience={params['patience']:.2f}min, "
            f"wants_table={'ANO' if params['wants_table'] else 'NE'}, "
            f"modifier={params['consumption_modifier']:.2f}×"
        )

    # Test objednávek
    print("\n🍽️ Test 5 objednávek:")

    for i in range(5):
        order = create_order_for_customer(config, ctype_id)
        items_names = [
            config.menu_items.__dict__.get(str(iid)).get_label() for iid in order
        ]
        print(
            f"   Zákazník {i+1}: {', '.join(items_names) if items_names else '(prázdná)'}"
        )

print("\n" + "=" * 80)
print("✅ TEST DOKONČEN")
print("=" * 80)